## Libraries

In [15]:
# libraries
import nltk #NLP lib
import numpy as np # numpy for arrays
import pandas as pd # pandas lib for data handling
from nltk.corpus import stopwords # common eng sstopwords
from nltk.stem import PorterStemmer #porter stemmer algo
from nltk.tokenize import sent_tokenize, word_tokenize
from bs4 import BeautifulSoup # HTML to txt

ps = PorterStemmer()
sw_nltk = stopwords.words('english') # bag of all common english stop words

## Supporting Functions

In [2]:
def TokStem(text,j): # Tokenise and Stem the given text
    words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final = []
    for w in words:
        final.append(ps.stem(w))
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I first met her she was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)

In [3]:
def hbt(text): # convert HTML body to text
    return BeautifulSoup(text).get_text()

In [47]:
def cos_sim(a,b): # finding cosine similarity score for given 2 strings
    # input two strings that are split into words
    us = [] # creating union set
    us += b
    for x in a:
        if x not in b:
            us.append(x)
    freqa,freqb,frequ = dict(),dict(),dict()
    wa,wb = [], []

    for word in a: # freq bag for string a
        if word in freqa:
            freqa[word] += 1
        else:
            freqa[word] = 1
    for word in b:# freq bag for string b
        if word in freqb:
            freqb[word] += 1
        else:
            freqb[word] = 1
    for word in us: # freq bag for union of a and b
        if word in frequ:
            frequ[word] += 1
        else:
            frequ[word] = 1
    
    for i in range(len(us)): # calc TitleVec for a and b
        x = us[i]
        if x in a:
            wa.append(freqa[x]/frequ[x])
        else:
            wa.append(0)
        if x in b:
            wb.append(freqb[x]/frequ[x])
        else:
            wb.append(0)
    wa = np.array(wa)
    wb = np.array(wb)
    return np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb)) # cosine value

## Loading the data 

In [4]:
df = pd.read_csv('../QueryResults.csv')
print(type(df))
print(df.columns)
# Q = df[['OId','OTitle','OBody','OTags']]
# print(Q.size)
# R = df[[ 'DId', 'DTitle', 'DBody', 'DTags']]
# P = Q.append(R,ignore_index=True)
# print(P.size)
# print(Q)
# df = df.dropna(axis = 0)
#print(df.describe())
# df.drop(['A'], axis = 1)
# print(df.head())
X_features = ['Title','Body','Tags','Topic']
# a = df.loc[10].at["OBody"]
#print(a)
#print(BeautifulSoup(a).get_text()) # html to text 

<class 'pandas.core.frame.DataFrame'>
Index(['OId', 'OTitle', 'OBody', 'OTags', 'DId', 'DTitle', 'DBody', 'DTags'], dtype='object')


## Data Preprocessing

In [21]:
k = 500
# duplicate questions for training (without sorting)
Q1 = df[['OId','OTitle','OBody','OTags']].loc[0:k-1].copy() # doop qns
Q2 = df[['DId','DTitle','DBody','DTags']].loc[0:k-1].copy() # original qns
print(Q1.shape)
te = []
for i in range(k): # pickout original qns
    x = int(Q2[['DId']].loc[i])
    if x in te:
        Q2.drop([i], inplace = True) # remove rows
    else:
        te.append(x)
# print(Q2.shape)
Q2 = Q2.reset_index(drop=True) # restore the indices
# print(Q2.loc[10])

# # process the body description wit tok, stem and export
for i in range(Q2.shape[0]):
    x = TokStem(hbt(Q2.loc[i,'DBody']),1)
    Q2.loc[i,'DBody'] = x
# Q2.to_csv('body_stem.csv') # export to CSV

(500, 4)


In [54]:
x = TokStem(hbt(df.loc[0,'OBody']),0)
y = TokStem(hbt(df.loc[0,'DBody']),0)
z = TokStem(hbt(df.loc[1,'OBody']),0)
print(x)
print(y)
print(z)
print(cos_sim(x,y))
print(cos_sim(z,y))
print(cos_sim(x,z))

['calcul', 'date', 'differ', 'two', 'date', 'years?', 'example:', '(datetime.now.today()', '-', '11/03/2007)', 'years.']
['given', 'datetim', 'repres', "person'", 'birthday,', 'calcul', 'age', 'years?']
['compar', 'age', 'birthdat', 'clause?', 'somth', 'like', 'code:', 'myrepository.where(x', '=>', 'fromag', '<=', 'x.birthdate.ag', '&&', 'x.birthdate.ag', '<=', 'toage)', '.select(x).tolist();', 'data', 'types:', 'fromage,', 'toag', ':', 'int', 'x.birthdate:', 'datetim']
0.21320071635561041
0.1414213562373095
0.0
